In [ ]:
import argparse #필요한 모듈 불러오기 
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

os.makedirs("images", exist_ok=True) #이미지를 넣을 디렉토리 만들기 

parser = argparse.ArgumentParser() #명령행 인수를 정의하고 구문 분석
parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training") #에폭 정의 
parser.add_argument("--batch_size", type=int, default=64, help="size of the batches") #배치 사이즈 정의 
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate") #학습률 정의 
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient") #Adam 옵티마이저의 하이퍼파라미터로, 그래디언트의 일차 모멘텀의 감쇠
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient") #그라디언트의 이차 모멘텀 감쇠를 나타냄 
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation") #배치 생성중에 사용할 CPU 스레드 수 
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space") #잠재 공간의 차원
parser.add_argument("--img_size", type=int, default=28, help="size of each image dimension") #이미지 사이즈 
parser.add_argument("--channels", type=int, default=1, help="number of image channels") #이미지의 채널 수 
parser.add_argument("--sample_interval", type=int, default=400, help="interval betwen image samples") #이미지 샘플 간의 간격. 프로그램이 훈련 중에 생성된 이미지의 샘플을 얼마나 자주 생성하고 저장 또는 표시해야 하는지를 결정하는 데 사용
opt = parser.parse_args() #이러한 인수의 값들을 받아서 
print(opt) #한꺼번에 출력

img_shape = (opt.channels, opt.img_size, opt.img_size) #이미지의 크기 정의

cuda = True if torch.cuda.is_available() else False #쿠다 설정


class Generator(nn.Module): #생성자 클래스 정의
    def __init__(self):
        super(Generator, self).__init__() #부모 클래스(nn.Module)의 생성자를 호출

        def block(in_feat, out_feat, normalize=True): #레이어를 정의하는 함수 block을 정의합니다. 이 함수는 입력 특징의 수, 출력 특징의 수, 정규화 여부를 받아 레이어 리스트를 반환
            layers = [nn.Linear(in_feat, out_feat)] #선형 레이어를 생성하고 리스트에 추가
            if normalize: #만약 normalize가 True이면, 배치 정규화 레이어를 생성하고 리스트에 추가
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True)) #Leaky ReLU 활성화 함수를 적용하는 레이어를 생성하고 리스트에 추가
            return layers

        self.model = nn.Sequential( #생성자 모델을 정의, sequential하게 만들어서 레이어층을 순차적으로 연결함 
            *block(opt.latent_dim, 128, normalize=False), #잠재 벡터 차원에서 시작해서 128차원으로 변환, 정규화 진행 x(배치 정규화임)
            *block(128, 256), #128 ->256
            *block(256, 512), #256 -> 512
            *block(512, 1024), #512 -> 1024
            nn.Linear(1024, int(np.prod(img_shape))), # 1024차원에서 이미지의 픽셀 수와 같은 차원으로 선형 변환
            nn.Tanh() # Tanh 활성화 함수를 적용하여 출력을 [-1, 1] 범위로 조정
        )

    def forward(self, z): #메서드는 주어진 잠재 벡터 z를 사용하여 모델을 통과
        img = self.model(z) #모델에 잠재 벡터를 전달하여 가짜 이미지를 생성
        img = img.view(img.size(0), *img_shape) #생성된 이미지를 원하는 이미지 모양으로 변환
        return img


class Discriminator(nn.Module): #판별자 클래스 정의 
    def __init__(self):
        super(Discriminator, self).__init__() #부모 클래스의 생성자를 호출함 

        self.model = nn.Sequential( #레이어 층을 순차적으로 연결 진행 
            nn.Linear(int(np.prod(img_shape)), 512), #모든 픽셀 차원을 평평하게 펼진다. 
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(), #활성화 함수를 마지막 단에 붙여 기울기 소실 문제를 해결하고 값을 0-1 범위로 조정하는 역할을 한다. 
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat) #평평하게  펼친 이미지를 모델에 전달하여 판별 결과를 출력함 

        return validity


# Loss function
adversarial_loss = torch.nn.BCELoss() #adversarial loss function을 정의함 

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Configure data loader
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader( #데이터 로더를 정의함 
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=opt.batch_size, #배치 사이즈 정의 
    shuffle=True, #무작위로 섞기 O 
)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

# ----------
#  Training
# ----------

for epoch in range(opt.n_epochs): #에폭 단위로 학습 되도록 반복문의 정의함 
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False) #ground truth 값을 정의함 
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

In [3]:
import torch.nn as nn

In [4]:
m = nn.AdaptiveAvgPool1d(5)

In [5]:
input = torch.randn(1, 64, 8)

In [6]:
output = m(input)

In [7]:
output.shape

torch.Size([1, 64, 5])